In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import random

In [2]:
# !rm data.csv

In [3]:
os.mkdir('../data_folder')

FileExistsError: [Errno 17] File exists: '../data_folder'

In [5]:
start = datetime.now()
if not os.path.isfile('../data_folder/data.csv'):
    data = open('../data_folder/data.csv', mode= 'w')
    
    row = list()
    files= ['../data/combined_data_1.txt', '../data/combined_data_2.txt', 
            '../data/combined_data_3.txt', '../data/combined_data_4.txt']
    
    for file in files:
        print("Reading ratings from {}...".format(file))
        with open(file) as f:
            for line in f:
                del row[:]
                line = line.strip()
                if line.endswith(':'):
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
        print("Done.\n")
    data.close()
print("Time taken: ", datetime.now()-start)            

Reading ratings from ../data/combined_data_1.txt...
Done.

Reading ratings from ../data/combined_data_2.txt...
Done.

Reading ratings from ../data/combined_data_3.txt...
Done.

Reading ratings from ../data/combined_data_4.txt...
Done.

Time taken:  0:02:56.755875


In [ ]:
print("creating the dataframe from data.csv file..")
df = pd.read_csv('../data_folder/data.csv', sep = ',', names = ['movies', 'user', 'rating', 'date'])
df.date = pd.to_datetime(df.date)
print("Done.\n")
print("Sorting the dataframe by date")
df.sort_values(by='date', inplace=True)
print("Done...")

creating the dataframe from data.csv file..


In [1]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.describe()['rating']

Checking for Nan Values

In [ ]:
print("No of Nan values in our dataframe: ", sum(df.isnull().any()))

Removing Duplicates

In [ ]:
dup_bool = df.duplicated(['movies', 'user', 'rating'])
dups = sum(dup_bool)
print("There are {} duplicate rating entries in data..".format(dups))

Basic Statistics

In [ ]:
print("Total Data ")
print("-"*50)
print("\nTotal no of ratings :", df.shape[0])
print("Total No of users     :", len(np.unique(df.user)))
print("Total No of movies     :", len(np.unique(df.movies)))

In [ ]:
if not os.path.isfile('train.csv'):
    df.iloc[:int(df.shape[0]*0.80)].to_csv("train.csv", index=False)
if not os.path.isfile('test.csv'):
    df.iloc[:int(df.shape[0]*0.80)].to_csv("test.csv", index=False)
train_df = pd.read_csv("train.csv", parse_dates=["date"])
test_df  = pd.read_csv("test.csv")

#Basic Statistics in Training Data

In [ ]:
print("Training Data ")
print("-"*50)
print("\nTotal no of ratings :", train_df.shape[0])
print("Total No of users     :", len(np.unique(train_df.user)))
print("Total No of movies     :", len(np.unique(train_df.movies)))

Basic Statistic in Test Data

In [ ]:
print("Training Data ")
print("-"*50)
print("\nTotal no of ratings :", test_df.shape[0])
print("Total No of users     :", len(np.unique(test_df.user)))
print("Total No of movies     :", len(np.unique(test_df.movies)))

EDA on Training Data

In [ ]:
 Distribution of Ratings

In [ ]:
def human(num, units = 'M'):
    units = units.lower()
    num = float(num)
    if units == 'k':
        return str(num/10**3) + " K"
    if units == 'm':
        return str(num/10**6) + " M"
    if units == 'b':
        return str(num/10**9) + " B"

In [ ]:
fig, ax = plt.subplots()
plt.title("Distribution of ratings over Training Dataset", fontsize=15)
sns.countplot(train_df.rating)
ax.set_yticklabels([human(item, 'M') for item in ax.get_yticks()])
ax.set_ylabel("No. of Ratings(Millions)")
plt.show()

Add new column (week day) to the data set for analysis

In [ ]:
pd.options.mode.chained_assignment = None
train_df['day_of_week'] = train_df.date.weekday_name
train_df.tail()

Number of Ratings per month

In [ ]:
ax = train_df>resample('m', on='date')['rating'].count().plot()
ax.set_title("No of ratings per month(Trainig Data)")
plt.xlabel("Month")
plt.ylabel("No. of ratings per month")
ax.set_yticklabels([human(item, 'M') for item in ax.get_yticks()])
plt.show()

No of rated movies per user

In [ ]:
no_of_rated_movies_per_user = train_df.groupby(by='user')['rating'].count()
no_of_rated_movies_per_user.head()

PDF and CDF for no_of_rated_movies_per_user

In [ ]:
fig = plt.figure(figsize=plt.figsize(.5))
ax1 = plt.subplot(121)
sns.kdeplot(no_of_rated_movies_per_user, shade=True, ax=ax1)
plt.xlabel("No of ratings by user")
plt.title("PDF")

ax2 = plt.subplot(122)
sns.kdeplot(no_of_rated_movies_per_user, shade=True,cumulative=True ax=ax2)
plt.xlabel("No of ratings by user")
plt.title("CDF")
plt.show()

In [ ]:
no_of_rated_movies_per_user.describe()